In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns

from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import os
print(os.listdir("../home/circleci/repo"))

['download_data.sh', '.DS_Store', 'requirements.txt', 'Makefile', 'test.csv', 'submit_data.sh', 'README.md', 'train.csv', '.ipynb_checkpoints', '.git', 'kernel.ipynb', '.circleci', 'gender_submission.csv', 'task_html.ipynb']


In [2]:
train = pd.read_csv('../home/circleci/repo/train.csv')
test = pd.read_csv('../home/circleci/repo/test.csv')
y_test_0 = pd.read_csv('../home/circleci/repo/gender_submission.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived 
sns.heatmap(train[["Survived","SibSp","Parch","Age","Fare"]].corr(),
                annot=True, fmt = ".2f", cmap = "coolwarm")

In [6]:
test = test.join(y_test_0.set_index('PassengerId'), on='PassengerId')
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188,0.363636
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576,0.481622
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,1.000000


**Preprocessing**

1. Combining train and test datasets to work with categirial features

In [9]:
data = pd.concat([train, test], ignore_index=True)
data.drop('PassengerId', 1, inplace=True)
X = data.drop('Survived', 1)
y = data['Survived']

In [10]:
data.shape

(1309, 11)

2. Drop raws with NaNs

In [11]:
data.dropna().shape

(270, 11)

*This method cant be use because we loose a huge ammount of data *

3. Numerical and categorial columns preprocessing

In [12]:
cat_cols = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
num_cols = list(set(X.columns.values.tolist()) - set(cat_cols))

In [13]:
X_num = X[num_cols].fillna(0)
X_cat = X[cat_cols].fillna('NA', axis=0).applymap(str)

In [14]:
#Categorial features one-hot encoding
encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())

#Numerical features scaling
scaler = StandardScaler()
scaler.fit(X_num, y)
X_num_scaled = scaler.transform(X_num)

#Combine features
X = np.hstack((X_cat_oh,X_num_scaled))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [15]:
print("Lenth of y:", len(y.values),'\n'"Lenth of X:", len(X))

Lenth of y: 1309 
Lenth of X: 1309


**Logistic regression**

In [16]:
y = np.reshape(y.values,(len(y),1))

In [17]:
(X_train, X_test, 
 y_train, y_test) = (X[:len(train)],  X[len(train):],
                    y[:len(train)],  y[len(train):])

In [18]:
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
cv = 3

clf = LR(random_state = 0)

grid_cv = GridSearchCV(clf, param_grid, cv = 3)
grid_cv.fit(X_train, y_train)

predicted = grid_cv.best_estimator_.predict_proba(X_test)[:,1]

roc_auc = roc_auc_score(y_test, predicted)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1255: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = None.
  " = {}.".format(self.n_jobs))
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1255: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = None.
  " = {}.".format(self.n_jobs))
/anaconda3/lib/python3.6/site-packages/s

In [19]:
roc_auc

0.98763355757815596

In [20]:
test_Survived = pd.Series(grid_cv.best_estimator_.predict(X_test), name="Survived")
result = pd.concat([y_test_0.PassengerId,test_Survived],axis=1)
result.to_csv("result.csv",index=False)

In [21]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
